In [ ]:
import requests
import pandas as pd

def fetch_mlb_player_data(season=2024):
    url = f"https://statsapi.mlb.com/api/v1/sports/1/players?season={season}"
    response = requests.get(url)
    response.raise_for_status()  # Raises HTTPError for bad responses
    data = response.json()
    return data.get("people", []), season

def parse_player_data(players, season):
    fields = [
        "id", "fullName", "firstName", "lastName", "primaryNumber", "birthDate", "currentAge",
        "birthCity", "birthStateProvince", "birthCountry", "height", "weight", "active",
        "currentTeam", "primaryPosition", "batSide", "pitchHand", "mlbDebutDate",
        "strikeZoneTop", "strikeZoneBottom"
    ]

    flat_players = []
    for player in players:
        flat_player = {field: player.get(field) for field in fields}
        if flat_player["currentTeam"]:
            flat_player["teamId"] = flat_player["currentTeam"].get("id")
            flat_player["teamName"] = flat_player["currentTeam"].get("name")
        if flat_player["primaryPosition"]:
            flat_player["position"] = flat_player["primaryPosition"].get("name")
        if flat_player["batSide"]:
            flat_player["batSide"] = flat_player["batSide"].get("description")
        if flat_player["pitchHand"]:
            flat_player["pitchHand"] = flat_player["pitchHand"].get("description")
        flat_player.pop("currentTeam", None)
        flat_player.pop("primaryPosition", None)
        flat_player["season"] = season
        flat_players.append(flat_player)

    return pd.DataFrame(flat_players)


In [8]:
# Example usage
players_raw, season = fetch_mlb_player_data(season=2024)
players_df = parse_player_data(players_raw, season)

# Show the first few rows
players_df



,id,fullName,firstName,lastName,primaryNumber,birthDate,currentAge,birthCity,birthStateProvince,birthCountry,...,active,batSide,pitchHand,mlbDebutDate,strikeZoneTop,strikeZoneBottom,teamId,teamName,position,season
0,671096,Andrew Abbott,Andrew,Abbott,41,1999-06-01,25,Lynchburg,VA,USA,...,True,Left,Left,2023-06-05,3.370,1.540,113,Cincinnati Reds,Pitcher,2024
1,682928,CJ Abrams,Paul,Abrams,5,2000-10-03,24,Alpharetta,GA,USA,...,True,Left,Right,2022-04-08,3.450,1.580,120,Washington Nationals,Shortstop,2024
2,650556,Bryan Abreu,Bryan,Abreu,52,1997-04-22,27,Santo Domingo Centro,None,Dominican Republic,...,True,Right,Right,2019-07-31,3.411,1.565,117,Houston Astros,Pitcher,2024
3,547989,José Abreu,José,Abreu,79,1987-01-29,38,Cienfuegos,None,Cuba,...,True,Right,Right,2014-03-31,3.420,1.550,117,Houston Astros,First Base,2024
4,677800,Wilyer Abreu,Wilyer,Abreu,52,1999-06-24,25,Maracaibo,None,Venezuela,...,True,Left,Left,2023-08-22,3.250,1.490,111,Boston Red Sox,Outfielder,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449,666171,Ryan Zeferjahn,Ryan,Zeferjahn,56,1998-02-28,27,Topeka,KS,USA,...,True,Right,Right,2024-08-25,3.575,1.681,108,Los Angeles Angels,Pitcher,2024
1450,672582,Angel Zerpa,Angel,Zerpa,61,1999-09-27,25,Valle de la Pascua,None,Venezuela,...,True,Left,Left,2021-09-30,3.371,1.535,118,Kansas City Royals,Pitcher,2024
1451,676604,Tyler Zuber,Tyler,Zuber,53,1995-06-16,29,White Hall,AR,USA,...,True,Right,Right,2020-07-24,3.319,1.513,139,Tampa Bay Rays,Pitcher,2024
1452,691172,Yosver Zulueta,Yosver,Zulueta,67,1998-01-23,27,Remedios,None,Cuba,...,True,Right,Right,2024-06-25,3.411,1.565,113,Cincinnati Reds,Pitcher,2024


In [10]:

players_df.to_csv("mlb_players_2024.csv", index=False)


